<a href="https://colab.research.google.com/github/javmencia/RelativeBelief/blob/BiasCalculations/RegressionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.special import gammaln
from scipy import stats
import time
import math

In [ ]:
# For x
mu_prior_mean = 0
mu_prior_sigma = 1
n=50
mu_0 = 0


p = 2

# For beta

beta_mu_0 = np.zeros((p, 1))
beta_cov_0 = np.eye(p)

# For sigma

sigma_mu_0 = 1
sigma_sigma_0 = 1

y_0 = np.ones(2)
with pm.Model() as m:

    # Define hyperparameters for all

    # For x
    mu_ = pm.Normal('mu_', mu=mu_prior_mean, sigma=mu_prior_sigma, shape=1)
    x_ = pm.Normal('x_', mu=mu_0, sigma=1, shape=n)

    # For beta and sigma
    beta = pm.MvNormal("beta", mu = beta_mu_0, cov = beta_cov_0, shape = (p, 1))
    sigma = pm.TruncatedNormal("sigma", mu = sigma_mu_0, sigma = sigma_sigma_0, lower = 0)

    y = pm.Normal('y', mu = pm.math.dot(x_, beta), sigma = sigma)

    # Priors
    prior_beta = pm.Deterministic("prior_beta", pm.logp(pm.MvNormal.dist(mu = beta_mu_0, cov = beta_cov_0, shape = (p, 1)), beta_mu_0))
    prior_sigma = pm.Deterministic("prior_sigma", pm.logp(pm.TruncatedNormal.dist(mu = sigma_mu_0, sigma = sigma_sigma_0, lower = 0), sigma_mu_0 ) )
    prior_y = pm.Deterministic("prior_y", pm.logp(pm.Normal.dist(mu = pm.math.dot(x_, beta_mu_0), sigma = sigma_mu_0), y_0))

    # Posteriors
    posterior_y = pm.Deterministic("posterior_y", pm.logp(pm.Normal.dist(mu = pm.math.dot(x_, beta), sigma = sigma), y_0))
    """
    # prior predictive data density evaluations
    prior_prior_pd = pm.Deterministic("prior_prior_pd", normal_logp(mu_0,0,1))

    posterior_var = 1/(1/mu_prior_sigma**2 + n/1)
    posterior_mean = (mu_prior_mean/mu_prior_sigma**2 + x_.sum()/1)*posterior_var
    posterior_prior_pd = pm.Deterministic("posterior_prior_pd",
                                          normal_logp(mu_0, posterior_mean, posterior_var**0.5))
    logBR_prior_pd = pm.Deterministic("logBR_prior_pd", posterior_prior_pd-prior_prior_pd) # pm.math.exp()

    likelihood_prior_pd = pm.Deterministic("likelihood_prior_pd", normal_logp(x_,mu_0,1).sum())"""

with m:
    idata = pm.sample(draws=1000)